# 2: Removing Outliers and Transforming data

## 2.1 Using Mahalanobis distance to identify outliers
As we are working with Multivariate data, it isn't the best practice for us to simply remove outliers based on a single variable. Thus, we utilise Mahalanobis distance to identify outliers based on multiple variables.

### 2.1.1: Loading and preparing dataset for mahalanobis distance calculation

In [1]:
import numpy as np
import pandas as pd
from scipy.spatial.distance import mahalanobis
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
import re
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler, MinMaxScaler


# Read the CSV file
raw_df = pd.read_csv('data/ResaleflatpricesbasedonregistrationdatefromJan2017onwards.csv')

# Load Dataset and perform simple transformations to numerical for mahalanobis distance
def convert_lease_to_months(lease_str):
    years, months = 0, 0  # Default values

    # Extract years
    year_match = re.search(r'(\d+)\s*years?', lease_str)
    if year_match:
        years = int(year_match.group(1))

    # Extract months
    month_match = re.search(r'(\d+)\s*months?', lease_str)
    if month_match:
        months = int(month_match.group(1))

    return years * 12 + months  # Convert to total months

# Apply function to column
raw_df["remaining_lease_months"] = raw_df["remaining_lease"].apply(convert_lease_to_months)

df = raw_df.drop_duplicates()
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 201581 entries, 0 to 201878
Data columns (total 12 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   month                   201581 non-null  object 
 1   town                    201581 non-null  object 
 2   flat_type               201581 non-null  object 
 3   block                   201581 non-null  object 
 4   street_name             201581 non-null  object 
 5   storey_range            201581 non-null  object 
 6   floor_area_sqm          201581 non-null  float64
 7   flat_model              201581 non-null  object 
 8   lease_commence_date     201581 non-null  int64  
 9   remaining_lease         201581 non-null  object 
 10  resale_price            201581 non-null  float64
 11  remaining_lease_months  201581 non-null  int64  
dtypes: float64(2), int64(2), object(8)
memory usage: 20.0+ MB


In [2]:
df_numeric = df[['floor_area_sqm', 'remaining_lease_months', 'resale_price']].copy() 


# Compute the mean and covariance matrix
mean_vec = np.mean(df_numeric, axis=0)
cov_matrix = np.cov(df_numeric, rowvar=False)

# Compute the inverse of the covariance matrix
inv_cov_matrix = np.linalg.inv(cov_matrix)

# Compute Mahalanobis distance for each data point
df_numeric['mahalanobis_dist'] = df_numeric.apply(lambda x: mahalanobis(x, mean_vec, inv_cov_matrix), axis=1)


# View top outliers
df_numeric.sort_values(by='mahalanobis_dist', ascending=False).head(70)

,floor_area_sqm,remaining_lease_months,resale_price,mahalanobis_dist
184423,366.7,564,1568000.0,11.685378
201510,94.0,1022,1518000.0,7.154778
198956,117.0,1038,1600000.0,7.072956
181248,112.0,1088,1588000.0,7.061830
199810,106.0,1020,1550000.0,7.031714
...,...,...,...,...
174275,112.0,1091,1450000.0,6.081288
187636,95.0,1089,1380000.0,6.071347
181331,94.0,1028,1365000.0,6.051205
165470,113.0,1135,1450000.0,6.026895


Based on the Mahalanobis Distance, we can see that there is a single outlier with a significantly different when compared to the other datapoints. Thus, we will remove this single outlier before we proceed.

In [3]:
# Find the index of the top outlier
top_outlier_idx = df_numeric['mahalanobis_dist'].idxmax()

# Remove the outlier from the dataset
df_cleaned = df.drop(index=top_outlier_idx)

# Verify the removal
print(f"Removed row at index: {top_outlier_idx}")
print(df_cleaned.shape)  # Check the new shape

# Load your dataset 
df_numeric_cleaned = df_cleaned[['floor_area_sqm', 'remaining_lease_months', 'resale_price']].copy() 

# Compute the mean and covariance matrix
mean_vec = np.mean(df_numeric_cleaned, axis=0)
cov_matrix = np.cov(df_numeric_cleaned, rowvar=False)

# Compute the inverse of the covariance matrix
inv_cov_matrix = np.linalg.inv(cov_matrix)

# Compute Mahalanobis distance for each data point
df_numeric_cleaned['mahalanobis_dist'] = df_numeric_cleaned.apply(lambda x: mahalanobis(x, mean_vec, inv_cov_matrix), axis=1)
# View top outliers
df_numeric_cleaned.sort_values(by='mahalanobis_dist', ascending=False).head(70)

Removed row at index: 184423
(201580, 12)


,floor_area_sqm,remaining_lease_months,resale_price,mahalanobis_dist
201510,94.0,1022,1518000.0,7.154761
198956,117.0,1038,1600000.0,7.072967
181248,112.0,1088,1588000.0,7.061823
199810,106.0,1020,1550000.0,7.031702
189021,113.0,1050,1580000.0,7.018959
...,...,...,...,...
187636,95.0,1089,1380000.0,6.071334
181331,94.0,1028,1365000.0,6.051192
165470,113.0,1135,1450000.0,6.026890
163278,113.0,1137,1450000.0,6.025885


## 2.2: Retaining only relevant columns

In [4]:
# relevant features

df_cleaned = df_cleaned[["month", 'town', 'storey_range', 'floor_area_sqm', 'flat_type','flat_model','remaining_lease','resale_price']]
df_cleaned.head()

,month,town,storey_range,floor_area_sqm,flat_type,flat_model,remaining_lease,resale_price
0,2017-01,ANG MO KIO,10 TO 12,44.0,2 ROOM,Improved,61 years 04 months,232000.0
1,2017-01,ANG MO KIO,01 TO 03,67.0,3 ROOM,New Generation,60 years 07 months,250000.0
2,2017-01,ANG MO KIO,01 TO 03,67.0,3 ROOM,New Generation,62 years 05 months,262000.0
3,2017-01,ANG MO KIO,04 TO 06,68.0,3 ROOM,New Generation,62 years 01 month,265000.0
4,2017-01,ANG MO KIO,01 TO 03,67.0,3 ROOM,New Generation,62 years 05 months,265000.0


## 2.3: Data cleaning for macrodata

### 2.3.1: Loading and preparing dataset for mahalanobis distance calculation

In [5]:
# Load the CSV File
df_macro = pd.read_csv('data/macro_data.csv')  # Read the CSV file into a DataFrame
df_macro.head()  # Display the first few rows of the DataFrame

,0,"Total Unemployment Rate, (SA)","Resident Unemployment Rate, (SA)","Citizen Unemployment Rate, (SA)",Household Net Worth,Assets,Financial Assets,Currency & Deposits,Shares & Securities,Listed Shares,...,Administrative & Support Services,Other Services Industries,Public Administration & Defence,Education,Health & Social Services,"Arts, Entertainment & Recreation",Other Services - Others,Ownership Of Dwellings,Gross Value Added At Basic Prices,Add: Taxes On Products
0,20244Q,1.9,2.8,2.9,3052040.2,3431440.9,1925062.2,658160.6,351132.6,150016.0,...,4034.1,13709.9,3130.8,3688.5,3766.2,1430.0,1679.6,5972.2,138399.2,7736.9
1,20243Q,1.9,2.6,2.7,2976551.1,3350578.6,1881260.9,633074.6,349577.9,152810.5,...,4250.7,13528.5,3111.0,3752.0,3782.5,1231.8,1635.7,5923.6,134988.6,7138.3
2,20242Q,2.0,2.7,2.8,2919647.9,3288024.2,1832632.8,629367.5,325669.0,145785.6,...,4158.1,13273.4,3094.5,3562.3,3718.9,1226.0,1652.2,5908.8,130579.4,6809.8
3,20241Q,2.1,2.9,3.1,2872703.3,3237721.6,1810903.7,623478.0,315652.4,137094.5,...,3940.9,13787.8,3296.6,3708.5,3524.7,1580.8,1650.7,5770.4,128816.3,6588.0
4,20234Q,2.0,2.7,2.9,2814508.3,3178795.1,1785432.6,611413.5,317090.2,134122.7,...,4057.0,13294.8,3096.3,3632.9,3546.8,1370.7,1630.8,5736.4,131754.9,7411.3


#### Segregate Data Quarterly: 
Group the data by quarters using pandas and verify the segregation.
Convert the data to appropriate format

In [6]:
# Convert '20241Q' format to datetime
def convert_quarter_to_date(quarter_str):
    year = int(quarter_str[:4])  # Extract the year
    quarter = int(quarter_str[4])  # Extract the quarter number
    month = (quarter - 1) * 3 + 1  # Calculate the starting month of the quarter
    return pd.Timestamp(year=year, month=month, day=1)

# Apply the conversion function to the 'Date' column
df_macro['0'] = df_macro['0'].apply(convert_quarter_to_date)

# Rename the column header 0 to 'Date'
df_macro.rename(columns={'0': 'Date'}, inplace=True)

# Display the cleaned DataFrame
print(df_macro)

         Date  Total Unemployment Rate, (SA)  \
0  2024-10-01                            1.9   
1  2024-07-01                            1.9   
2  2024-04-01                            2.0   
3  2024-01-01                            2.1   
4  2023-10-01                            2.0   
5  2023-07-01                            2.0   
6  2023-04-01                            1.9   
7  2023-01-01                            1.8   
8  2022-10-01                            2.0   
9  2022-07-01                            2.1   
10 2022-04-01                            2.1   
11 2022-01-01                            2.3   
12 2021-10-01                            2.4   
13 2021-07-01                            2.6   
14 2021-04-01                            2.7   
15 2021-01-01                            2.9   
16 2020-10-01                            3.2   
17 2020-07-01                            3.4   
18 2020-04-01                            3.0   
19 2020-01-01                           

Extrapolate to monthly data

In [7]:
# Convert 'Date' column to datetime
df_macro['Date'] = pd.to_datetime(df_macro['Date'])

# Extrapolate to monthly data
monthly_data = []
for _, row in df_macro.iterrows():
    start_date = row['Date']
    row_data = row.to_dict()  # Convert the row to a dictionary
    for i in range(3):  # Repeat for 3 months
        month_date = start_date + pd.DateOffset(months=i)
        new_row = row_data.copy()  # Copy the row data
        new_row['Date'] = month_date  # Update the date for the new row
        monthly_data.append(new_row)

# Create a new DataFrame with monthly data
monthly_df = pd.DataFrame(monthly_data)

# Sort the DataFrame by date
monthly_df = monthly_df.sort_values(by='Date').reset_index(drop=True)

# Display the resulting DataFrame
print(monthly_df)

         Date  Total Unemployment Rate, (SA)  \
0  2017-01-01                            2.3   
1  2017-02-01                            2.3   
2  2017-03-01                            2.3   
3  2017-04-01                            2.2   
4  2017-05-01                            2.2   
..        ...                            ...   
91 2024-08-01                            1.9   
92 2024-09-01                            1.9   
93 2024-10-01                            1.9   
94 2024-11-01                            1.9   
95 2024-12-01                            1.9   

    Resident Unemployment Rate, (SA)  Citizen Unemployment Rate, (SA)  \
0                                3.3                              3.5   
1                                3.3                              3.5   
2                                3.3                              3.5   
3                                3.1                              3.3   
4                                3.1                      

Using Mahalanobis distance to identify outliers (same as above)

In [8]:
df_numeric = monthly_df.drop(columns=['Date'])  # Drop the 'Date' column for analysis
# Compute the mean and covariance matrix
mean_vec = np.mean(df_numeric, axis=0)
cov_matrix = np.cov(df_numeric, rowvar=False)

# Compute the pseudo-inverse of the covariance matrix
inv_cov_matrix = np.linalg.pinv(cov_matrix)

# Compute Mahalanobis distance for each data point
df_numeric['mahalanobis_dist'] = df_numeric.apply(lambda x: mahalanobis(x, mean_vec, inv_cov_matrix), axis=1)


# View top outliers
df_numeric.sort_values(by='mahalanobis_dist', ascending=False).head(70)

,"Total Unemployment Rate, (SA)","Resident Unemployment Rate, (SA)","Citizen Unemployment Rate, (SA)",Household Net Worth,Assets,Financial Assets,Currency & Deposits,Shares & Securities,Listed Shares,Unlisted Shares,...,Other Services Industries,Public Administration & Defence,Education,Health & Social Services,"Arts, Entertainment & Recreation",Other Services - Others,Ownership Of Dwellings,Gross Value Added At Basic Prices,Add: Taxes On Products,mahalanobis_dist
14,2.1,2.9,3.0,1775409.9,2118140.1,1200736.5,427806.6,210837.2,95659.3,31674.2,...,12357.9,2860.5,3454.6,2904.6,1655.9,1484.3,5103.6,108882.0,7268.3,5.53869
12,2.1,2.9,3.0,1775409.9,2118140.1,1200736.5,427806.6,210837.2,95659.3,31674.2,...,12357.9,2860.5,3454.6,2904.6,1655.9,1484.3,5103.6,108882.0,7268.3,5.53869
13,2.1,2.9,3.0,1775409.9,2118140.1,1200736.5,427806.6,210837.2,95659.3,31674.2,...,12357.9,2860.5,3454.6,2904.6,1655.9,1484.3,5103.6,108882.0,7268.3,5.53869
60,2.3,3.0,3.2,2454819.4,2813474.3,1629850.0,562648.0,272949.4,104050.6,34176.8,...,12473.8,3244.3,3529.3,3283.8,952.2,1412.1,5413.9,123458.3,6425.7,5.53869
61,2.3,3.0,3.2,2454819.4,2813474.3,1629850.0,562648.0,272949.4,104050.6,34176.8,...,12473.8,3244.3,3529.3,3283.8,952.2,1412.1,5413.9,123458.3,6425.7,5.53869
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51,2.7,3.6,3.8,2308910.2,2652921.4,1559956.4,528895.6,265522.0,98413.3,31608.8,...,11781.0,3065.7,3379.6,3219.4,750.9,1312.5,5503.9,118445.3,6559.3,5.53869
46,3.2,4.3,4.4,2176938.1,2509941.9,1483708.1,517148.8,237750.0,90237.7,27736.9,...,11885.4,3044.8,3404.7,3158.3,926.8,1306.0,5434.8,114269.7,7705.0,5.53869
47,3.2,4.3,4.4,2176938.1,2509941.9,1483708.1,517148.8,237750.0,90237.7,27736.9,...,11885.4,3044.8,3404.7,3158.3,926.8,1306.0,5434.8,114269.7,7705.0,5.53869
45,3.2,4.3,4.4,2176938.1,2509941.9,1483708.1,517148.8,237750.0,90237.7,27736.9,...,11885.4,3044.8,3404.7,3158.3,926.8,1306.0,5434.8,114269.7,7705.0,5.53869


(same steps as above for df_cleaned)

In [9]:
# Find the index of the top outlier
top_outlier_idx = df_numeric['mahalanobis_dist'].idxmax()

# Remove the outlier from the dataset
df_cleaned_macro = df_macro.drop(index=top_outlier_idx)

# Verify the removal
print(f"Removed row at index: {top_outlier_idx}")
print(df_cleaned.shape)  # Check the new shape

# Load your dataset 
df_numeric_cleaned = df_cleaned_macro.drop(columns=['Date'])  # Drop 'Date' column for numeric analysis

# Compute the mean and covariance matrix
mean_vec = np.mean(df_numeric_cleaned, axis=0)
cov_matrix = np.cov(df_numeric_cleaned, rowvar=False)

# Compute the pseudo-inverse of the covariance matrix
inv_cov_matrix = np.linalg.pinv(cov_matrix)

# Compute Mahalanobis distance for each data point
df_numeric_cleaned['mahalanobis_dist'] = df_numeric_cleaned.apply(lambda x: mahalanobis(x, mean_vec, inv_cov_matrix), axis=1, result_type='expand')

# View top outliers
df_numeric_cleaned.sort_values(by='mahalanobis_dist', ascending=False).head(70)

Removed row at index: 12
(201580, 8)


,"Total Unemployment Rate, (SA)","Resident Unemployment Rate, (SA)","Citizen Unemployment Rate, (SA)",Household Net Worth,Assets,Financial Assets,Currency & Deposits,Shares & Securities,Listed Shares,Unlisted Shares,...,Other Services Industries,Public Administration & Defence,Education,Health & Social Services,"Arts, Entertainment & Recreation",Other Services - Others,Ownership Of Dwellings,Gross Value Added At Basic Prices,Add: Taxes On Products,mahalanobis_dist
22,2.3,3.2,3.3,1913684.3,2253573.0,1296945.9,461905.5,208221.7,90674.7,31675.2,...,12370.6,2842.3,3437.6,3037.3,1448.5,1610.9,5382.8,112867.0,7283.5,5.388159
23,2.2,3.1,3.3,1870684.1,2213274.8,1264247.0,455015.2,197449.6,89190.7,31406.3,...,12589.1,2971.2,3433.5,3015.8,1581.6,1590.5,5346.2,110476.6,7527.1,5.388159
11,2.3,3.0,3.2,2454819.4,2813474.3,1629850.0,562648.0,272949.4,104050.6,34176.8,...,12473.8,3244.3,3529.3,3283.8,952.2,1412.1,5413.9,123458.3,6425.7,5.388159
21,2.2,3.1,3.2,1945769.1,2284216.3,1312298.6,466414.1,204066.0,87630.8,30844.1,...,12519.9,2794.1,3480.3,3128.9,1516.6,1612.4,5423.6,113293.9,7161.2,5.388159
7,1.8,2.6,2.7,2656059.7,3014560.0,1692158.1,589556.9,280796.7,118418.4,34130.1,...,13226.8,3284.2,3655.4,3405.2,1276.1,1566.7,5505.4,124661.2,6466.2,5.388159
4,2.0,2.7,2.9,2814508.3,3178795.1,1785432.6,611413.5,317090.2,134122.7,35118.4,...,13294.8,3096.3,3632.9,3546.8,1370.7,1630.8,5736.4,131754.9,7411.3,5.388159
6,1.9,2.7,2.9,2700112.9,3059937.5,1722043.4,606246.1,281825.5,118074.8,34227.3,...,12913.9,3092.0,3491.0,3560.0,1171.7,1576.5,5633.5,126370.6,6415.3,5.388159
24,2.2,3.0,3.1,1825181.2,2169731.7,1222766.2,440065.0,189931.9,85561.7,31460.5,...,12153.8,2808.8,3415.1,2991.0,1397.8,1544.1,5257.9,113290.8,7180.6,5.388159
9,2.1,2.9,3.0,2532728.2,2896129.7,1635380.0,569251.0,265943.8,101263.1,34040.2,...,12748.2,3099.6,3562.3,3446.2,1180.8,1431.7,5525.0,125453.4,6417.3,5.388159
1,1.9,2.6,2.7,2976551.1,3350578.6,1881260.9,633074.6,349577.9,152810.5,37024.7,...,13528.5,3111.0,3752.0,3782.5,1231.8,1635.7,5923.6,134988.6,7138.3,5.388159


## Join both datasets

In [10]:
# Convert both columns to datetime
df_cleaned['month'] = pd.to_datetime(df_cleaned['month'], format='%Y-%m')  # Convert '2017-01' to datetime
df_cleaned_macro['Date'] = pd.to_datetime(df_cleaned_macro['Date'], format='%Y-%m-%d')  # Convert '2017-01-01' to datetime

# Standardize the format (truncate df2['date'] to year-month)
df_cleaned_macro['Date'] = df_cleaned_macro['Date'].dt.to_period('M').dt.to_timestamp()  # Convert to '2017-01'

# Merge the DataFrames
merged_df = pd.merge(df_cleaned, df_cleaned_macro, left_on='month', right_on='Date', how='inner')

# Convert the 'Date' column (or any datetime column) back to string format
merged_df['month'] = merged_df['month'].dt.strftime('%Y-%m')  # Specify the desired format, e.g., 'YYYY-MM-DD'

# Display the merged DataFrame
print(merged_df)



         month        town storey_range  floor_area_sqm  flat_type  \
0      2017-01  ANG MO KIO     10 TO 12            44.0     2 ROOM   
1      2017-01  ANG MO KIO     01 TO 03            67.0     3 ROOM   
2      2017-01  ANG MO KIO     01 TO 03            67.0     3 ROOM   
3      2017-01  ANG MO KIO     04 TO 06            68.0     3 ROOM   
4      2017-01  ANG MO KIO     01 TO 03            67.0     3 ROOM   
...        ...         ...          ...             ...        ...   
64568  2024-10      YISHUN     01 TO 03           145.0  EXECUTIVE   
64569  2024-10      YISHUN     01 TO 03           146.0  EXECUTIVE   
64570  2024-10      YISHUN     10 TO 12           152.0  EXECUTIVE   
64571  2024-10      YISHUN     01 TO 03           169.0  EXECUTIVE   
64572  2024-10      YISHUN     10 TO 12           142.0  EXECUTIVE   

           flat_model     remaining_lease  resale_price       Date  \
0            Improved  61 years 04 months      232000.0 2017-01-01   
1      New Generati

## 2.4: Transforming Datatypes

In [11]:
df = merged_df.copy()  
df.dtypes

month                                            object
town                                             object
storey_range                                     object
floor_area_sqm                                  float64
flat_type                                        object
                                                 ...   
            Arts, Entertainment & Recreation    float64
            Other Services - Others             float64
    Ownership Of Dwellings                      float64
    Gross Value Added At Basic Prices           float64
    Add: Taxes On Products                      float64
Length: 64, dtype: object

### 2.4.1: Changing month to months from Jan 2017

In [12]:
# changing month to months from jan 2017
df[['y', 'm_from2017']]=df['month'].str.split(pat="-", expand = True)
df['y'] = pd.to_numeric(df['y']).sub(2017).mul(12)
df['m_from2017'] = pd.to_numeric(df['m_from2017'])
df['m_from2017'] = df['m_from2017'].add(df['y'])
# can delete col y now

### 2.4.2: Changing remaining lease to no. of months left in lease

In [13]:
# changing remaining lease to no of months left
def convert_to_months(lease_str):
    years = 0
    months = 0
    
    # Extract years and months using regex
    year_match = re.search(r'(\d+) years?', lease_str)
    month_match = re.search(r'(\d+) months?', lease_str)

    if year_match:
        years = int(year_match.group(1))
    if month_match:
        months = int(month_match.group(1))

    return (years * 12) + months

# Apply the function to transform the column
df['lease_months_left'] = df['remaining_lease'].apply(convert_to_months)
# remove remaining_lease

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64573 entries, 0 to 64572
Data columns (total 67 columns):
 #   Column                                                                            Non-Null Count  Dtype         
---  ------                                                                            --------------  -----         
 0   month                                                                             64573 non-null  object        
 1   town                                                                              64573 non-null  object        
 2   storey_range                                                                      64573 non-null  object        
 3   floor_area_sqm                                                                    64573 non-null  float64       
 4   flat_type                                                                         64573 non-null  object        
 5   flat_model                                                  

### 2.4.3: Encoding towns using one hot encoding

In [15]:
# encoding the towns - one hot encoding

# Standardize column names
df.rename(columns=lambda x: x.strip().lower(), inplace=True)

# Ensure 'town' column is a string and clean spaces
df['town'] = df['town'].astype(str).str.strip()

# One-hot encode
df_encoded = pd.get_dummies(df, columns=['town'], dtype = int)

# Ensure only specified towns are kept
towns = [
    "JURONG WEST", "SENGKANG", "WOODLANDS", "PUNGGOL", "TAMPINES", "YISHUN", "BEDOK", "HOUGANG",
    "ANG MO KIO", "BUKIT MERAH", "CHOA CHU KANG", "TOA PAYOH", "BUKIT BATOK", "BUKIT PANJANG",
    "KALLANG/WHAMPOA", "PASIR RIS", "GEYLANG", "QUEENSTOWN", "SEMBAWANG", "JURONG EAST",
    "BISHAN", "CLEMENTI", "SERANGOON", "CENTRAL AREA", "MARINE PARADE", "BUKIT TIMAH"
]

town_columns = [f'town_{town}' for town in towns if f'town_{town}' in df_encoded.columns]
df_encoded = df_encoded[['town'] + town_columns] if 'town' in df_encoded.columns else df_encoded[town_columns]

df = pd.concat([df, df_encoded], axis = 1)
df.head()

,month,town,storey_range,floor_area_sqm,flat_type,flat_model,remaining_lease,resale_price,date,"total unemployment rate, (sa)",...,town_GEYLANG,town_QUEENSTOWN,town_SEMBAWANG,town_JURONG EAST,town_BISHAN,town_CLEMENTI,town_SERANGOON,town_CENTRAL AREA,town_MARINE PARADE,town_BUKIT TIMAH
0,2017-01,ANG MO KIO,10 TO 12,44.0,2 ROOM,Improved,61 years 04 months,232000.0,2017-01-01,2.3,...,0,0,0,0,0,0,0,0,0,0
1,2017-01,ANG MO KIO,01 TO 03,67.0,3 ROOM,New Generation,60 years 07 months,250000.0,2017-01-01,2.3,...,0,0,0,0,0,0,0,0,0,0
2,2017-01,ANG MO KIO,01 TO 03,67.0,3 ROOM,New Generation,62 years 05 months,262000.0,2017-01-01,2.3,...,0,0,0,0,0,0,0,0,0,0
3,2017-01,ANG MO KIO,04 TO 06,68.0,3 ROOM,New Generation,62 years 01 month,265000.0,2017-01-01,2.3,...,0,0,0,0,0,0,0,0,0,0
4,2017-01,ANG MO KIO,01 TO 03,67.0,3 ROOM,New Generation,62 years 05 months,265000.0,2017-01-01,2.3,...,0,0,0,0,0,0,0,0,0,0


### 2.4.4: Encoding flat model using one hot encoding

In [16]:
# encoding the flat_model - one hot encoding
#### still need to edit this chunk to adapt it

# Standardize column names
df.rename(columns=lambda x: x.strip().lower(), inplace=True)

# Ensure 'town' column is a string and clean spaces
df['flat_model'] = df['flat_model'].astype(str).str.strip()

# One-hot encode
df_encoded = pd.get_dummies(df, columns=['flat_model'], dtype = int)

# Ensure only specified towns are kept
models = [
    "Model A", "Improved", "New Generation", "Premium Apartment", "Simplified", "Apartment", "Maisonette", "Standard",
    "DBSS", "Model A2", "Model A-Maisonette", "Adjoined flat", "Type S1", "2-room",
    "Type S2", "Premium Apartment Loft", "Terrace", "Multi Generation", "3Gen", "Improved-Maisonette",
    "Premium Maisonette"
]

model_columns = [f'flat_model_{model}' for model in models if f'flat_model_{model}' in df_encoded.columns]
df_encoded = df_encoded[['flat_model'] + model_columns] if 'flat_model' in df_encoded.columns else df_encoded[model_columns]

df = pd.concat([df, df_encoded], axis = 1)
df.tail()

,month,town,storey_range,floor_area_sqm,flat_type,flat_model,remaining_lease,resale_price,date,"total unemployment rate, (sa)",...,flat_model_Adjoined flat,flat_model_Type S1,flat_model_2-room,flat_model_Type S2,flat_model_Premium Apartment Loft,flat_model_Terrace,flat_model_Multi Generation,flat_model_3Gen,flat_model_Improved-Maisonette,flat_model_Premium Maisonette
64568,2024-10,YISHUN,01 TO 03,145.0,EXECUTIVE,Maisonette,62 years 09 months,888888.0,2024-10-01,1.9,...,0,0,0,0,0,0,0,0,0,0
64569,2024-10,YISHUN,01 TO 03,146.0,EXECUTIVE,Maisonette,62 years 10 months,848000.0,2024-10-01,1.9,...,0,0,0,0,0,0,0,0,0,0
64570,2024-10,YISHUN,10 TO 12,152.0,EXECUTIVE,Maisonette,59 years 11 months,867000.0,2024-10-01,1.9,...,0,0,0,0,0,0,0,0,0,0
64571,2024-10,YISHUN,01 TO 03,169.0,EXECUTIVE,Apartment,66 years 09 months,860000.0,2024-10-01,1.9,...,0,0,0,0,0,0,0,0,0,0
64572,2024-10,YISHUN,10 TO 12,142.0,EXECUTIVE,Apartment,62 years 02 months,980000.0,2024-10-01,1.9,...,0,0,0,0,0,0,0,0,0,0


### 2.4.5: Encoding Flat Type using label encoding

In [17]:
# encoding the flat type - label encoding
flat_type_mapping = {
    '1 ROOM': 0,
    '2 ROOM': 1,
    '3 ROOM': 2,
    '4 ROOM': 3,
    '5 ROOM': 4,
    'EXECUTIVE': 5,
    'MULTI-GENERATION': 6
}

# Apply the mapping to the flat_type column
df['flat_type'] = df['flat_type'].map(flat_type_mapping)
df.tail()


,month,town,storey_range,floor_area_sqm,flat_type,flat_model,remaining_lease,resale_price,date,"total unemployment rate, (sa)",...,flat_model_Adjoined flat,flat_model_Type S1,flat_model_2-room,flat_model_Type S2,flat_model_Premium Apartment Loft,flat_model_Terrace,flat_model_Multi Generation,flat_model_3Gen,flat_model_Improved-Maisonette,flat_model_Premium Maisonette
64568,2024-10,YISHUN,01 TO 03,145.0,5,Maisonette,62 years 09 months,888888.0,2024-10-01,1.9,...,0,0,0,0,0,0,0,0,0,0
64569,2024-10,YISHUN,01 TO 03,146.0,5,Maisonette,62 years 10 months,848000.0,2024-10-01,1.9,...,0,0,0,0,0,0,0,0,0,0
64570,2024-10,YISHUN,10 TO 12,152.0,5,Maisonette,59 years 11 months,867000.0,2024-10-01,1.9,...,0,0,0,0,0,0,0,0,0,0
64571,2024-10,YISHUN,01 TO 03,169.0,5,Apartment,66 years 09 months,860000.0,2024-10-01,1.9,...,0,0,0,0,0,0,0,0,0,0
64572,2024-10,YISHUN,10 TO 12,142.0,5,Apartment,62 years 02 months,980000.0,2024-10-01,1.9,...,0,0,0,0,0,0,0,0,0,0


### 2.4.6: Encoding Storey Range using Label encoding

In [18]:
# encoding the storey_range - label encoding
storey_range_mapping = {
    '01 TO 03': 0, '04 TO 06': 1, '07 TO 09': 2, '10 TO 12': 3, '13 TO 15': 4,
    '16 TO 18': 5, '19 TO 21': 6, '22 TO 24': 7, '25 TO 27': 8, '28 TO 30': 9,
    '31 TO 33': 10, '34 TO 36': 11, '37 TO 39': 12, '40 TO 42': 13, '43 TO 45': 14,
    '46 TO 48': 15, '49 TO 51': 16
}

# Apply the mapping to the storey_range column
df['storey_range'] = df['storey_range'].map(storey_range_mapping)
df.tail()

,month,town,storey_range,floor_area_sqm,flat_type,flat_model,remaining_lease,resale_price,date,"total unemployment rate, (sa)",...,flat_model_Adjoined flat,flat_model_Type S1,flat_model_2-room,flat_model_Type S2,flat_model_Premium Apartment Loft,flat_model_Terrace,flat_model_Multi Generation,flat_model_3Gen,flat_model_Improved-Maisonette,flat_model_Premium Maisonette
64568,2024-10,YISHUN,0,145.0,5,Maisonette,62 years 09 months,888888.0,2024-10-01,1.9,...,0,0,0,0,0,0,0,0,0,0
64569,2024-10,YISHUN,0,146.0,5,Maisonette,62 years 10 months,848000.0,2024-10-01,1.9,...,0,0,0,0,0,0,0,0,0,0
64570,2024-10,YISHUN,3,152.0,5,Maisonette,59 years 11 months,867000.0,2024-10-01,1.9,...,0,0,0,0,0,0,0,0,0,0
64571,2024-10,YISHUN,0,169.0,5,Apartment,66 years 09 months,860000.0,2024-10-01,1.9,...,0,0,0,0,0,0,0,0,0,0
64572,2024-10,YISHUN,3,142.0,5,Apartment,62 years 02 months,980000.0,2024-10-01,1.9,...,0,0,0,0,0,0,0,0,0,0


### 2.4.7: Encoding macro_data

In [19]:
# Separate categorical and numerical columns
categorical_cols = df.select_dtypes(include=['object']).columns
numerical_cols = df.select_dtypes(include=['float64', 'int64']).columns

# 1. One-Hot Encoding for Categorical Features
df_encoded = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

# 2. Standardization for Numerical Features
scaler = StandardScaler()
df_encoded[numerical_cols] = scaler.fit_transform(df_encoded[numerical_cols])

# 3. (Optional) Normalization for Numerical Features
scaler = MinMaxScaler()
df_encoded[numerical_cols] = scaler.fit_transform(df_encoded[numerical_cols])

# Display the encoded DataFrame
print(df_encoded.head())

   storey_range  floor_area_sqm  flat_type  resale_price       date  \
0        0.1875        0.061321   0.166667      0.057792 2017-01-01   
1        0.0000        0.169811   0.333333      0.070478 2017-01-01   
2        0.0000        0.169811   0.333333      0.078935 2017-01-01   
3        0.0625        0.174528   0.333333      0.081049 2017-01-01   
4        0.0000        0.169811   0.333333      0.081049 2017-01-01   

   total unemployment rate, (sa)  resident unemployment rate, (sa)  \
0                         0.3125                              0.35   
1                         0.3125                              0.35   
2                         0.3125                              0.35   
3                         0.3125                              0.35   
4                         0.3125                              0.35   

   citizen unemployment rate, (sa)  household net worth  assets  ...  \
0                              0.4                  0.0     0.0  ...   
1       

### 2.4.8: Normalising the floor area column

In [20]:
# scaler = MinMaxScaler()
# df[['floor_area_sqm']] = scaler.fit_transform(df[['floor_area_sqm']])

# # Normalising the values within the lease_months_left column

### 2.4.9: Removing redundant columns used for transformation

In [21]:
#remove redundant/encoded cols
df.drop(columns=['remaining_lease', 'y', 'town','flat_model', 'month'], inplace=True)

In [22]:
df.dtypes

storey_range                               int64
floor_area_sqm                           float64
flat_type                                  int64
resale_price                             float64
date                              datetime64[ns]
                                       ...      
flat_model_Terrace                         int64
flat_model_Multi Generation                int64
flat_model_3Gen                            int64
flat_model_Improved-Maisonette             int64
flat_model_Premium Maisonette              int64
Length: 109, dtype: object

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64573 entries, 0 to 64572
Columns: 109 entries, storey_range to flat_model_Premium Maisonette
dtypes: datetime64[ns](1), float64(56), int64(52)
memory usage: 53.7 MB


In [28]:
df.to_csv('data/data_macro.csv', index = False)

## 2.5: Exporting transformed data with test train split to new csv

In [24]:
from sklearn.model_selection import train_test_split

# Split the data into training (80%), validation (10%), and test (10%)
train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42)
valid_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

# Optionally, you can print the shapes of the resulting datasets
print("Training set shape:", train_df.shape)
print("Validation set shape:", valid_df.shape)
print("Test set shape:", test_df.shape)

Training set shape: (51658, 109)
Validation set shape: (6457, 109)
Test set shape: (6458, 109)


In [25]:
# normalise the splitted data
cols_to_norm = ['storey_range', 'floor_area_sqm', 'flat_type', 'm_from2017', 'lease_months_left']

# Initialize the MinMaxScaler
scaler = MinMaxScaler()

# Fit the scaler only on the training data
scaler.fit(train_df[cols_to_norm])

# Apply the transformation (without refitting) to all splits
train_df[cols_to_norm] = scaler.transform(train_df[cols_to_norm])
test_df[cols_to_norm] = scaler.transform(test_df[cols_to_norm])
valid_df[cols_to_norm] = scaler.transform(valid_df[cols_to_norm])
display(train_df)

,storey_range,floor_area_sqm,flat_type,resale_price,date,"total unemployment rate, (sa)","resident unemployment rate, (sa)","citizen unemployment rate, (sa)",household net worth,assets,...,flat_model_Adjoined flat,flat_model_Type S1,flat_model_2-room,flat_model_Type S2,flat_model_Premium Apartment Loft,flat_model_Terrace,flat_model_Multi Generation,flat_model_3Gen,flat_model_Improved-Maisonette,flat_model_Premium Maisonette
9621,0.0000,0.287736,0.500000,375000.0,2018-07-01,2.1,2.8,3.0,1823899.1,2168119.5,...,0,0,0,0,0,0,0,0,0,0
2132,0.1250,0.160377,0.333333,340000.0,2017-04-01,2.2,3.1,3.3,1682447.3,2012412.8,...,0,0,0,0,0,0,0,0,0,0
20740,0.0625,0.287736,0.500000,443888.0,2019-10-01,2.2,3.2,3.2,1998097.3,2337418.2,...,0,0,0,0,0,0,0,0,0,0
61717,0.1250,0.066038,0.166667,365000.0,2024-07-01,1.9,2.6,2.7,2976551.1,3350578.6,...,0,0,0,0,0,0,0,0,0,0
25908,0.2500,0.457547,0.833333,565000.0,2020-07-01,3.4,4.6,4.7,2104813.9,2434122.4,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62570,0.0625,0.287736,0.500000,482000.0,2024-10-01,1.9,2.8,2.9,3052040.2,3431440.9,...,0,0,0,0,0,0,0,0,0,0
38158,0.1875,0.396226,0.666667,500000.0,2022-01-01,2.3,3.0,3.2,2454819.4,2813474.3,...,0,0,0,0,0,0,0,0,0,0
860,0.1250,0.278302,0.500000,530000.0,2017-01-01,2.3,3.3,3.5,1658881.3,1985339.9,...,0,0,0,0,0,0,0,0,0,0
15795,0.1250,0.349057,0.500000,420000.0,2019-04-01,2.3,3.2,3.3,1913684.3,2253573.0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
# generate the new csv files to use for the model
train_df.to_csv('data/train_data_macro.csv', index = False)
valid_df.to_csv('data/valid_data_macro.csv', index = False)
test_df.to_csv('data/test_data_macro.csv', index = False)